In [2]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
import torch

/home/shashank/miniconda3/envs/tag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sentence_transformer_model = SentenceTransformer('all-mpnet-base-v2')

In [8]:
data = pd.read_csv("data/papers.csv")
# data = data[data["Abstract"].notna()]

# add a ai_generated_tag column
data["ai_generated_Methods_tag"] = ""
data["ai_generated_Highlevel_tag"] = ""
data["ai_generated_Morphologies_tag"] = ""

In [11]:
all_Methods_tags_dict = {
"Deep reinforcement learning is a machine learning approach that combines deep neural networks with reinforcement learning (RL) algorithms to enable agents to learn optimal actions through trial and error in complex environments. This paper trains a deep reinforcement learning policy for a task.":"Deep reinforcement learning",

"Informative path planning is a robotics and AI technique that aims to find optimal paths for agents or vehicles by considering the most informative locations or viewpoints along the way to gather valuable data or observations.":"Informative path planning",

"Simulation is the imitation or emulation of the operation of a real-world process or system over time, allowing for analysis, testing, and understanding of its behavior without directly interacting with the actual system. This paper proposes a method to model systems to provide better simulations. We showcase our simulation.":"Simulation",

# "By learning the parameters of the simulator, we aim to improve the accuracy of simulations so that they closely match real-world dynamics, which is crucial for the development of control and perception methods in robotics. ": "Learning for simulation",

"Localization in robotics refers to the process of determining a robot's precise position and orientation within its environment, typically using sensors and mapping techniques to navigate and perform tasks accurately.":"Localization",

"Mapping in robotics refers to the process of creating a spatial representation of an environment using sensors and data, enabling a robot to understand and navigate its surroundings.":"Mapping",

"simultaneous localization and mapping is a computational technique used in robotics to enable a robot to build a map of its environment while simultaneously determining its own position within that map in real-time.":"SLAM",

"Multi robot simultaneous localization and mapping in robotics is the collaborative process where multiple robots work together to create a map of their environment while simultaneously determining their own positions within that map.":"Multi-robot SLAM",

"Multi robot coordination and planning in robotics refers to the process of enabling multiple robots to work together efficiently and collaboratively to achieve common goals by optimizing their actions and movements in a coordinated manner.":"Multi-robot coordination and planning",

"Deformable manipulation in robotics refers to the capability of a robot to interact with and manipulate objects that have flexible or deformable properties, such as textiles or soft materials, by adapting its grasp and manipulation strategies to accommodate their changing shapes.":"Deformable manipulation",

"Learning from demonstrations in robotics is a process where a robot acquires new skills or behaviors by observing and imitating human or expert demonstrations.":"Learning from demonstrations",

"Motion planning in robotics is the process of determining a sequence of feasible movements for a robot to navigate from its current position to a desired goal while avoiding obstacles and adhering to constraints.":"Motion planning",

"Meta learning in robotics and machine learning is a technique where a model learns how to learn, enabling it to adapt and generalize to new tasks or environments more efficiently by leveraging prior learning experiences.":"Meta learning",
}

all_method_tags = list(all_Methods_tags_dict.keys())
method_tags_embeddings = sentence_transformer_model.encode(all_method_tags, convert_to_tensor=True)

In [16]:
all_Highlevel_tags_dict = {
    "This paper proposes a learning based method. Learning in robotics refers to the ability of a robot or robotic system to acquire knowledge, adapt to its environment, and improve its performance over time through various methods such as machine learning, reinforcement learning, or other artificial intelligence techniques. This can involve tasks like recognizing objects, understanding speech, or navigating unfamiliar terrain by learning from past experiences or data.": "Learning",
    "This paper proposes a planning method. Planning in robotics involves the process of generating a sequence of actions or a trajectory that a robot should follow to achieve a specific goal or task. This can include path planning (finding the best route from one point to another), motion planning (determining how to move robot joints or actuators to reach a desired configuration), or task planning (sequencing a series of actions to complete a complex task efficiently).": "Planning",
    "This paper proposes a novel control algorithm. Control in robotics is the mechanism or set of algorithms that enable a robot to manipulate its actuators (such as motors or servos) to achieve desired movements or behaviors. This involves regulating the robot's state, ensuring it follows a prescribed path or trajectory, and reacting to disturbances in its environment to maintain stability and accuracy. Control can be open-loop (predefined actions) or closed-loop (responsive to feedback).": "Control",
    "Our methods propose novel dynamics for systems. Dynamics in robotics refers to the study of the motion and forces involved in the movement of a robot or its components. This includes understanding how forces and torques affect the robot's motion and how the robot's mass, inertia, and geometry influence its behavior. Dynamics plays a crucial role in tasks such as robot design, motion planning, and control, as it helps predict and model how a robot will respond to different inputs and external forces.": "Dynamics",
}
all_highlevel_tags = list(all_Highlevel_tags_dict.keys())
highlevel_tags_embeddings = sentence_transformer_model.encode(all_highlevel_tags, convert_to_tensor=True)

In [ ]:
all_Morphologies_tags_dict = {
    "Aquatic robots have a morphology designed for operation in water environments. They are typically equipped with features such as buoyancy control, underwater propulsion systems (e.g., fins, propellers), and sensors suited for underwater navigation and data collection. Aquatic robots can include submersibles, underwater drones, and remotely operated vehicles (ROVs), among others..": "Aquatic",
    "Aerial UAVs have a morphology optimized for flying in the air. They are characterized by their wings, rotors, or other means of generating lift and control. Aerial UAVs are used for various applications, including surveillance, aerial photography, mapping, and environmental monitoring.": "Aerial UAVs",
    "Ground robots are designed to operate on the terrestrial surface, such as roads, rough terrain, or indoor environments. They typically have wheels, tracks, or legs for mobility, and their sensors and control systems are tailored for ground-based tasks. Examples of ground robots include autonomous cars, robotic vacuum cleaners, and search-and-rescue robots.": "Ground",
    "Robots with manipulator arms have a morphology that includes one or more articulated arms equipped with joints, end-effectors (such as grippers or tools), and sensors. These robots are designed for tasks that require precise manipulation, such as pick-and-place operations, assembly, and handling objects in unstructured environments.": "Manipulator Arms",
    "Mobile manipulator arms combine the features of both mobile platforms (wheels, tracks, or legs for movement) and manipulator arms. They are capable of navigating in their environment while also performing manipulation tasks. Mobile manipulator arms are versatile and can be used in applications like warehouse automation, agriculture, and disaster response.": "Mobile Manipulator Arms",
}

In [24]:
threshold = 0.2

for i, row in data.iterrows():
    abstract = row["Abstract"]
    if pd.isna(abstract):
        continue

    # abstract_sentences = abstract.split(".")
    abstract_sentences = [abstract]

    # Compute embedding for both abstract and tags
    abstract_embedding = sentence_transformer_model.encode(abstract_sentences, convert_to_tensor=True)

    method_cosine_scores = util.dot_score(method_tags_embeddings, abstract_embedding)
    highlevel_cosine_scores = util.dot_score(highlevel_tags_embeddings, abstract_embedding)
    

    # cosine_scores = cosine_scores.max(1).values
    method_cosine_scores = method_cosine_scores.mean(1)
    # Sort the scores for tags
    sorted_indices = method_cosine_scores.argsort(descending=True)
    # get the tags with scores above the threshold
    sorted_indices = sorted_indices[method_cosine_scores[sorted_indices] > threshold]#[0:1]
    # find top tags that stand out
    normalized_method_scores = (method_cosine_scores - method_cosine_scores.mean()) / method_cosine_scores.std()
    # pick scores that are above 1 standard deviations
    sorted_indices = sorted_indices[normalized_method_scores[sorted_indices] > 1]
    top_5_method_tags_dict = {all_Methods_tags_dict[all_method_tags[idx]]: str(normalized_method_scores[idx].item())[:4] for idx in sorted_indices}


    highlevel_cosine_scores = highlevel_cosine_scores.mean(1)

    # get the tag with the highest score
    sorted_indices = highlevel_cosine_scores.argsort(descending=True)[0:1]
    # find top tags that stand out
    top_highlevel_tag = all_highlevel_tags[sorted_indices[0]]
    top_highlevel_score = str(highlevel_cosine_scores[sorted_indices[0]].item())[:4]
    top_5_highlevel_tags_dict = {all_Highlevel_tags_dict[top_highlevel_tag]: top_highlevel_score}


    title = row["Title"]
    # if len(sorted_indices) < 2:
        # print("here")
    # print("\nTitle: {}: tags: {}".format(title, top_5_method_tags_dict))

    if len(top_5_method_tags_dict) == 0:
        print("No method tags found for this paper")
        continue
    if len(top_5_highlevel_tags_dict) == 0:
        print("No highlevel tags found for this paper")
        continue

    # add the tags to the dataframe
    list_of_method_tags = list(top_5_method_tags_dict.keys())
    list_of_highlevel_tags = list(top_5_highlevel_tags_dict.keys())
    data.at[i, "ai_generated_Methods_tag"] = ", ".join(list_of_method_tags)
    data.at[i, "ai_generated_Highlevel_tag"] = ", ".join(list_of_highlevel_tags)

No method tags found for this paper
No method tags found for this paper
No method tags found for this paper
No method tags found for this paper
No method tags found for this paper
No method tags found for this paper
No method tags found for this paper
No method tags found for this paper
No method tags found for this paper
No method tags found for this paper
No method tags found for this paper
No method tags found for this paper
No method tags found for this paper
No method tags found for this paper
No method tags found for this paper
No method tags found for this paper
No method tags found for this paper
No method tags found for this paper


In [25]:
data[["Title", "ai_generated_Methods_tag", "ai_generated_Highlevel_tag"]]

,Title,ai_generated_Methods_tag,ai_generated_Highlevel_tag
0,GranularGym: High Performance Simulation for R...,"Simulation, Deep reinforcement learning",Dynamics
1,IndustReal: Transferring Contact-Rich Assembly...,"Deep reinforcement learning, Learning from dem...",Learning
2,Fast and Scalable Signal Inference for Active ...,"Informative path planning, Motion planning",Planning
3,Language-Informed Transfer Learning for Embodi...,"Deep reinforcement learning, Meta learning",Learning
4,RREx-BoT: Remote Referring Expressions with a ...,Meta learning,Learning
...,...,...,...
462,The Design and Control of a Prototype Quadrupe...,"Motion planning, Multi-robot coordination and ...",Control
463,Mobility evaluation of a wheeled microrover us...,"Motion planning, Multi-robot coordination and ...",Planning
464,Mission Reachability for Extraterrestrial Rovers.,"Motion planning, Mapping",Planning
465,On the Development of EMG Control for a Prosth...,"Learning from demonstrations, Meta learning",Control


In [ ]:
# save the dataframe
data.to_csv("data/papers_with_ai_tags.csv", index=False)